# Árboles binarios

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
from sklearn.datasets import make_friedman1
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#Lectura de base de datos
M = pd.read_csv("C:\\Users\\ricardo\\Desktop\\ArbolesBinarios\\EjemploArbolBinario_2D.csv",dtype='str',encoding = "ISO-8859-1")
M[["Y","X1","X2"]] = M[["Y","X1","X2"]].astype(float)
M.head()

In [ ]:
#Función error cuadrático medio.
def Fun_ECM(v1,v2):
    suma, n = 0, len(v1)
    for i in range(n):
        suma = suma + (v1[1]-v2[1])**2
    return (suma**0.5)/n

# Construcción del árbol binario

In [ ]:
#Transformación de los datos en un arreglo de numpy.
X_train = np.asarray(M[["X1","X2"]].copy(deep=True).reset_index(drop=True))
X_test = np.asarray(M[["X1","X2"]].copy(deep=True).reset_index(drop=True))
y_train = np.asarray(M[["Y"]].copy(deep=True).reset_index(drop=True))
y_test = np.asarray(M[["Y"]].copy(deep=True).reset_index(drop=True))

In [ ]:
#Creación de la clase nodo
class TNode:
    def __init__(self, depth, X, y):
        self.depth = depth
        self.X = X #Matriz de variables explicativas
        self.y = y #Matriz de variables de respuesta
        #Inicialización de parámetros de split
        self.j = None #Coordenada para realizar la partición
        self.xi = None #Valor de partición dentro de la coordenada
        #Inicialización de un hijo vacío
        self.left = None #Posteriormente en la función Construct_Subtree se define aquí un árbol
        self.right = None #Posteriormente en la función Construct_Subtree se define aquí un árbol
        #Inicialización del predictor del nodo
        self.g = None
    def CalculateLoss(self):
        if(len(self.y)==0):
            return 0
        else:
            return np.sum(np.power(self.y - self.y.mean(),2))
treeRoot = TNode(0, X_train,y_train)
print("Profundida del árbol",treeRoot.depth)
print("Variables explicativas del árbol",treeRoot.X)
print("Variable de respuesta del árbol",treeRoot.y)


In [ ]:
#Observación sobe localización de valores en un Data Frame
print(X_train[:,0]) #Se regresa la primer columna de la matriz
print(X_train[:,1]) #Se regresa la segunda columna de la matriz
ids_bis = X_train[:,0]<=6
print(ids_bis)

In [ ]:
#Función de split
def DataSplit(X,y,j,xi):
    ids = X[:,j]<=xi #X[:,j] es un arreglo formado por las entradas j de cada vector del arreglo original
    Xt = X[ids == True, :] #Elementos del arreglo original que cumplen idf
    Xf = X[ids == False, :]
    yt = y[ids == True]
    yf = y[ids == False]
    return Xt, yt, Xf, yf
#Ejemplo
Xt, yt, Xf, yf = DataSplit(X_train,y_train,1,15)
print(Xt)
print(yt)

In [ ]:
#Ejemplo de función shape.
m, n = X_train.shape
print("Número de renglones",m)
print("Número de columnas",n)

In [ ]:
#Función split óptimo, sólo en el caso inicial
def CalculateOptimalSplit(node):
    X = node.X
    y = node.y
    best_var = 0 #Dimensión en la que se relizará la partición
    best_xi = X[0,best_var] #Valor en cada coordenada para relizar la división de la región factible
    best_split_val = node.CalculateLoss()
    m, n = X.shape
    for j in range(0,n):
        for i in range(0,m):
            xi = X[i,j]
            Xt, yt, Xf, yf = DataSplit(X,y,j,xi)
            tmpt = TNode(0, Xt, yt)
            tmpf = TNode(0, Xf, yf)
            loss_t = tmpt.CalculateLoss()
            loss_f = tmpf.CalculateLoss()
            curr_val = loss_t + loss_f
            if (curr_val < best_split_val):
                best_split_val = curr_val
                best_var = j
                best_xi = xi
    return best_var, best_xi #Coordenada de partición, valor para la partición (notar que es un valor de la muestra)
#Ejemplo
best_var, best_xi = CalculateOptimalSplit(treeRoot)
print("Coordenada óptima para hacer la primer partición de la región factible",best_var)
print("Valor óptimo de la coordenada para hacer la patición",best_xi)

In [ ]:
#Ejemplo función recursiva
def f_factorial(n):
    if n == 0 or n == 1:
        y = 1
    else:
        y = n*f_factorial(n-1)
    return y
#Ejemplo
n = 5
print("El factorial del número ",n," es igual a ",f_factorial(n))

In [ ]:
#Construcción del subárbol
def Construct_Subtree(node, max_depth):
    if(node.depth == max_depth or len(node.y) == 1): #El valor 1 es arbitrario para detener el algoritmo e indicar un número mínimo de valores para promediar
        node.g = node.y.mean() #Aquí va la función que regresa el valor del árbol binario
    else:
        j, xi = CalculateOptimalSplit(node)
        node.j = j #Coordenda para realizar la partición
        node.xi = xi #Valor de la coordenada para realizar la partición
        Xt, yt, Xf, yf = DataSplit(node.X, node.y, j, xi)
        if(len(yt)>0):
            node.left = TNode(node.depth+1,Xt,yt) #Se agrega un nodo derecho al nodo anterior
            Construct_Subtree(node.left, max_depth) #Función recursiva
        if(len(yf)>0):
            node.right = TNode(node.depth+1, Xf,yf) #Se agrega un nodo derecho al nodo izquierdo
            Construct_Subtree(node.right, max_depth) #Función recursiva
    return node
#Ejemplo
maxdepth = 2
T = Construct_Subtree(treeRoot, maxdepth)

In [ ]:
#Ejemplo de visualización del árbol binario construido
print("************************Nivel 0************************")
print("Nivel actual",T.depth)
print("Coordenada para clasificar",T.j)
print("Valor de la coordenada para clasificar",T.xi)
print("************************Nivel 1 izquierdo************************")
N1I = T.left
print("Nivel actual",N1I.depth)
print("Coordenada para clasificar",N1I.j)
print("Valor de la coordenada para clasificar",N1I.xi)
print("************************Nivel 1 derecho************************")
N1D = T.right
print("Nivel actual",N1D.depth)
print("Coordenada para clasificar",N1D.j)
print("Valor de la coordenada para clasificar",N1D.xi)
print("************************Nivel 2 izquierdo - izquierdo************************")
N2II = N1I.left
print("Nivel actual",N2II.depth)
print("Coordenada para clasificar",N2II.j)
print("Valor de la coordenada para clasificar",N2II.xi)
print("************************Nivel 2 izquierdo - derecho************************")
N2ID = N1I.right
print("Nivel actual",N2ID.depth)
print("Coordenada para clasificar",N2ID.j)
print("Valor de la coordenada para clasificar",N2ID.xi)
print("************************Nivel 2 derecho - izquierdo************************")
N2DI = N1D.left
print("Nivel actual",N2DI.depth)
print("Coordenada para clasificar",N2DI.j)
print("Valor de la coordenada para clasificar",N2DI.xi)
print("************************Nivel 2 derecho - derecho************************")
N2DD = N1D.right
print("Nivel actual",N2DD.depth)
print("Coordenada para clasificar",N2DD.j)
print("Valor de la coordenada para clasificar",N2DD.xi)

In [ ]:
def Predict(X,node):
    if(node.right == None and node.left != None):
        return Predict(X,node.left)
    if(node.right != None and node.left == None):
        return Predict(X,node.right)
    if(node.right == None and node.left == None):
        return node.g
    else:
        if(X[node.j] <= node.xi): #Notar que se da como parámetro un renglon de características (no la matriz completa)
            return Predict(X,node.left)
        else :
            return Predict(X,node.right)
#Ejemplo
y_hat = np.zeros(len(X_test))
for i in range(len(X_test)):
    y_hat[i] = Predict(X_test[i],treeRoot)
ECM1 = Fun_ECM(y_hat,y_test)
print("Error cuadrático medio = ", ECM1)
#Definición de Data Frame para el comparativo de los resultados
MComp1 = pd.DataFrame(index=range(len(y_hat)),columns=["Datos verdaderos","Datos aproximados"])
MComp1["Datos verdaderos"] = y_test
MComp1["Datos aproximados"] = y_hat
MComp1

In [ ]:
#Ejemplo de una predicción particular
C_1 = Predict([1,3],treeRoot)
C_2 = Predict([7,4],treeRoot)
C_3 = Predict([2,8],treeRoot)
C_4 = Predict([9,10],treeRoot)
print("Pronóstico en el primer cuadrante ",C_1)
print("Pronóstico en el segundo cuadrante ",C_2)
print("Pronóstico en el tercer cuadrante ",C_3)
print("Pronóstico en el cuarto cuadrante ",C_4)

# Cálculo mediante funciones definidas en Python

In [ ]:
#Comparativo con el método de sklearn
#Fuente: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
from sklearn.tree import DecisionTreeRegressor
regTree = DecisionTreeRegressor(max_depth = 2)
regTree.fit(X_train,y_train)
y_hat2 = regTree.predict(X_test)
ECM2 = Fun_ECM(y_hat2,y_test)
print("Error cuadrático medio = ", ECM2)
#Definición de Data Frame para el comparativo de los resultados
MComp2 = pd.DataFrame(index=range(len(y_hat)),columns=["Datos verdaderos","Datos aproximados"])
MComp2["Datos verdaderos"] = y_test
MComp2["Datos aproximados"] = y_hat2
MComp2